In [3]:
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
noise_power_density = 5.0119e-21
W = 10e6 # bandwidth (unit:Hz)
circuit_power = 1 # constant circuit power of transmitter (unit: Watt)

In [5]:
def spectral_efficiency(P, h, N0, bandwidth):
    """
    Calculate the total spectral efficiency for two transmitters.
    Parameters:
    - P: Array of transmit powers [P1, P2]
    - h: Array of channel gains [h11, h12, h21, h22] (first number is index of tx, second is index of rx)
    - N0: Noise power density
    - W: Bandwidth

    Returns:
    - Total spectral efficiency
    """
    P1, P2 = P
    # h11, h12, h21, h22 = h
    h11, h12, h21, h22 = h

    SE1 = np.log2(1 + h11 * P1 / (N0 * bandwidth + h21 * P2))
    SE2 = np.log2(1 + h22 * P2 / (N0 * bandwidth + h12 * P1))

    total_SE = SE1 + SE2

    return total_SE

In [6]:
def energy_efficiency(P, h, N0, bandwidth, Pc):
    """
    Calculate the energy efficiency for two transmitters.

    Parameters:
    - P: Array of transmit powers [P1, P2]
    - h: Array of channel gains [h11, h12, h21, h22]
    - N0: Noise power density
    - W: Bandwidth
    - Pc: Constant circuit power of the transmitter

    Returns:
    - Energy efficiency (EE)
    """
    h11, h12, h21, h22 = h
    P1, P2 = P

    # Calculate the spectral efficiencies for the given power configuration
    SE1 = np.log2(1 + h11 * P1 / (N0 * bandwidth + h21 * P2))
    SE2 = np.log2(1 + h22 * P2 / (N0 * bandwidth + h12 * P1))

    P1, P2 = P

    EE = SE1/(P1+Pc) + SE2/(P2+Pc)

    return EE

In [4]:
# Read json files of generated responses
file_name_ee = "generated_responses_for_energy_efficiency.json"
file_name_se = "generated_responses_for_spectral_efficiency.json"

with open(file_name_ee, 'r') as f:
    ee_data = json.load(f)

with open(file_name_se, 'r') as f:
    se_data = json.load(f)

In [7]:
# Lists to store energy and spectral efficiency for original and predicted responses
original_energy_efficiencies = []
predicted_energy_efficiencies = []
original_spectral_efficiencies = []
predicted_spectral_efficiencies = []

In [17]:
# Process each entry in the ee dataset
for entry in ee_data:
    # Convert A values from string to integers
    channel_gain_values = [int(x) for x in entry['input'].replace('If A is', '').replace(',', '').split()]
    # Convert B values from string to integers
    original_power_values = [int(x.strip().rstrip('.')) for x in entry['original_response'].replace('B is', '').split(',')]
    predicted_power_values = [int(x.strip().rstrip('.')) for x in entry['predicted_response'].replace('B is', '').split(',')]
    
    # # Calculate efficiencies for original response
    original_energy = energy_efficiency(original_power_values, channel_gain_values, noise_power_density, W, circuit_power)
    original_energy_efficiencies.append(original_energy)
    
    # # Calculate efficiencies for predicted response
    predicted_energy = energy_efficiency(predicted_power_values, channel_gain_values, noise_power_density, W, circuit_power)
    predicted_energy_efficiencies.append(predicted_energy)

C:\Users\PMYLS\AppData\Local\Temp\ipykernel_5080\4205438493.py:20: RuntimeWarning: invalid value encountered in log2
  SE2 = np.log2(1 + h22 * P2 / (N0 * bandwidth + h12 * P1))
C:\Users\PMYLS\AppData\Local\Temp\ipykernel_5080\4205438493.py:19: RuntimeWarning: invalid value encountered in log2
  SE1 = np.log2(1 + h11 * P1 / (N0 * bandwidth + h21 * P2))
C:\Users\PMYLS\AppData\Local\Temp\ipykernel_5080\4205438493.py:20: RuntimeWarning: divide by zero encountered in log2
  SE2 = np.log2(1 + h22 * P2 / (N0 * bandwidth + h12 * P1))
C:\Users\PMYLS\AppData\Local\Temp\ipykernel_5080\4205438493.py:19: RuntimeWarning: divide by zero encountered in log2
  SE1 = np.log2(1 + h11 * P1 / (N0 * bandwidth + h21 * P2))


In [ ]:
# Calculate average efficiencies
avg_original_energy = sum(original_energy_efficiencies) / len(original_energy_efficiencies)
avg_predicted_energy = sum(predicted_energy_efficiencies) / len(predicted_energy_efficiencies)

avg_original_spectral = sum(original_spectral_efficiencies) / len(original_spectral_efficiencies)
avg_predicted_spectral = sum(predicted_spectral_efficiencies) / len(predicted_spectral_efficiencies)